<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_AtividadePriscila_Solvers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install ortools

In [ ]:
from google.colab import files
uploaded = files.upload() # subir arquivo Instancia

Saving Instancia100.txt to Instancia100.txt


In [ ]:
# colab: incluir !pip install
import numpy as np # módulo para manipulação de vetores e matrizes
from ortools.linear_solver import pywraplp  # CBC
# import docplex.mp.sdetails                  # CPLEX
# from docplex.mp.model import Model          # CPLEX
from ortools.sat.python import cp_model     # CPSAT

def leitura(arquivo):
    arq = open(arquivo, "r") # r- read
    lixo = arq.readline()  # linha com texto Number of jobs
    linha = arq.readline() # linha com número de tarefas
    N = int(linha)         # linha com o número de tarefas
    p = np.zeros(N, dtype=np.int64)  # p = [p0, p1,..., p(N-1)]
    w = np.zeros(N, dtype=np.int64)
    d = np.zeros(N, dtype=np.int64)
    lixo = arq.readline()  # linha com texto Job data (job number, release date, processing time, due date)
    for i in range(N):  # repetir N vezes
        # i-ésima linha
        linha = arq.readline().split()  # posições: 0-indice 1-p, 2-w, 3-d
        p[i] = int(linha[1])
        w[i] = int(linha[2])
        d[i] = int(linha[3])
    return N, p, w, d   

def modelo_Min_TerminoPonderado_CPLEX(N, p, w, d):
    # cria o modelo
    mdl = Model(name="Única Máquina")

    M = 2*sum(p)

    # alocação de memória para as estruturas do problema
    x = [0]*N  
    y = []
    for i in range(N):
        crialinha = [0]*N
        y.append(crialinha)

    # declaração das variáveis de decisão  
    for i in range(N):     # min, max 
        x[i] = mdl.integer_var(0, None, 'x'+str(i))  

    for i in range(N):
        for k in range(N):
            if k > i :            
                y[i][k] = mdl.binary_var('y'+str(i)+","+str(k))

    # declaração das restrições 
    for i in range(N):
        mdl.add_constraint( x[i] + p[i] <= d[i] )
        for k in range(N):
            if k > i :
                mdl.add_constraint( x[i] + p[i] <= x[k] + M*(1-y[i][k]) )
                mdl.add_constraint( x[k] + p[k] <= x[i] + M*y[i][k] )

    # função objetivo
    mdl.minimize( mdl.sum(w[i]*(x[i] + p[i]) for i in range(N)) )
    
    # limitar tempo de execução (em segundos)
    mdl.set_time_limit(1*60*60) 
    
    #resolver o problema
    mdl.solve()
    status = mdl.solve_details.status
    print("Status: ", status)
        
    print("Número de Variáveis: ", mdl.number_of_variables)  
    print("Número de Restrições: ", mdl.number_of_constraints)
    print("Tempo de execução (s): %.4f" %mdl.solve_details.time)
    print("Nós B&B: ", mdl.solve_details.nb_nodes_processed)
        
    print("Função-objetivo: ", mdl.objective_value)
    print("Best Bound: %.4f" %mdl.solve_details.best_bound)            
    print("GAP: %.4f" %mdl.solve_details.mip_relative_gap)   
    
    print("Lista de Alocação (tarefa, início):")
    aux = []        
    for i in range(N):
        aux.append((i, x[i].solution_value))        
    solOrd = sorted(aux, key=lambda tup: tup[1])
    print(solOrd)
    print()
    
def modelo_Min_TerminoPonderado_CBC(N, p, w, d):
    # cria o modelo
    mdl = pywraplp.Solver('Única Máquina', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    M = 2*sum(p)

    # alocação de memória para as estruturas do problema
    x = [0]*N  
    y = []
    for i in range(N):
        crialinha = [0]*N
        y.append(crialinha)

    # declaração das variáveis de decisão    
    for i in range(N):     # min, max 
        x[i] = mdl.IntVar(0, mdl.infinity(), 'x'+str(i))  

    for i in range(N):
        for k in range(N):
            if k > i :            
                y[i][k] = mdl.IntVar(0, 1,'y'+str(i)+","+str(k))

    # declaração das restrições 
    for i in range(N):
        mdl.Add( x[i] + p[i] <= d[i] )
        for k in range(N):
            if k > i :
                mdl.Add( x[i] + p[i] <= x[k] + M*(1-y[i][k]) )
                mdl.Add( x[k] + p[k] <= x[i] + M*y[i][k] )

    # função objetivo
    mdl.Minimize( mdl.Sum(w[i]*(x[i] + p[i]) for i in range(N)) )
    
    # limitar tempo de execução (em milisegundos)
    mdl.SetTimeLimit(1*60*60*1000)   # 1 hora
    
    #resolver o problema
    status = mdl.Solve()
    print("Status: ", status)

    print("Número de Variáveis: ", mdl.NumVariables())  
    print("Número de Restrições: ", mdl.NumConstraints())
    print("Tempo de execução (s): %.4f" %float(mdl.WallTime()/1000))
    print("Nós B&B: ", mdl.nodes())
    
    print("Função-objetivo: ", mdl.Objective().Value())
    print("Best Bound: %.4f" %mdl.Objective().BestBound())
    
    gap = -1
    if mdl.Objective().Value():
        gap = (mdl.Objective().Value() - mdl.Objective().BestBound())/mdl.Objective().Value()
    print("GAP: %.4f" %gap)
    
    print("Lista de Alocação (tarefa, início):")
    aux = []
    for i in range(N):
        aux.append((i, x[i].solution_value()))
    solOrd = sorted(aux, key=lambda tup: tup[1])
    print(solOrd)
    print()
    
def modelo_Min_TerminoPonderado_CPSAT(N, p, w, d):
    # cria o modelo
    mdl = cp_model.CpModel()

    M = int(2*sum(p))

    # alocação de memória para as estruturas do problema
    x = [0]*N  
    y = []
    for i in range(N):
        crialinha = [0]*N
        y.append(crialinha)

    xmax = int(sum(p))
    # declaração das variáveis de decisão    
    for i in range(N):     # min, max 
        x[i] = mdl.NewIntVar(0, xmax, 'x'+str(i))  

    for i in range(N):
        for k in range(N):
            if k > i :            
                y[i][k] = mdl.NewBoolVar('y'+str(i)+","+str(k))

    # declaração das restrições 
    for i in range(N):
        mdl.Add( x[i] + p[i] <= d[i] )
        for k in range(N):
            if k > i :
                mdl.Add( x[i] + p[i] <= x[k] + M*(1-y[i][k]) )
                mdl.Add( x[k] + p[k] <= x[i] + M*y[i][k] )

    # função objetivo
    mdl.Minimize( sum(w[i]*(x[i] + p[i]) for i in range(N)) )
    
    # resolver com CP-SAT
    solver = cp_model.CpSolver()
    # núcleos para processamento
    solver.parameters.num_search_workers = 8
    # limitar tempo de execução (em segundos)
    solver.parameters.max_time_in_seconds = 1*60*60
    status = solver.Solve(mdl)
    status = solver.StatusName(status)
    print("Status: ", status)
    
    z = solver.ObjectiveValue()
    bb = solver.BestObjectiveBound()
    nos = solver.NumBranches()
    gap = -1
    if solver.ObjectiveValue():
        gap = (solver.ObjectiveValue() - solver.BestObjectiveBound())/solver.ObjectiveValue()
    print("Função-objetivo: ", z)
    print("Best Bound: %.4f" %bb)
    print("Nós B&B: ", nos)
    print("GAP: %.4f" %gap)
    print(solver.ResponseStats())    
    
    print("Lista de Alocação (tarefa, início):")
    aux = []
    for i in range(N):
        aux.append((i, solver.Value(x[i])))
    solOrd = sorted(aux, key=lambda tup: tup[1])
    print(solOrd)
    print()

In [ ]:
# *********************** 10 TAREFAS CP-SAT ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia10_.txt")  

print("===== CP-SAT =====")
modelo_Min_TerminoPonderado_CPSAT(N, p, w, d)

===== CP-SAT =====
Status:  OPTIMAL
Função-objetivo:  17481.0
Best Bound: 17481.0000
Nós B&B:  316
GAP: 0.0000
CpSolverResponse summary:
status: OPTIMAL
objective: 17481
best_bound: 17481
booleans: 198
conflicts: 192
branches: 316
propagations: 4448
integer_propagations: 5461
restarts: 294
lp_iterations: 0
walltime: 0.035658
usertime: 0.035658
deterministic_time: 0.00401596
primal_integral: 0.00220992

Lista de Alocação (tarefa, início):
[(2, 0), (7, 79), (9, 133), (3, 198), (0, 253), (5, 306), (6, 379), (4, 507), (1, 644), (8, 785)]



In [ ]:
# *********************** 20 TAREFAS CP-SAT ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("Instancia20.txt")  

print("===== CP-SAT =====")
modelo_Min_TerminoPonderado_CPSAT(N, p, w, d)

===== CP-SAT =====
Status:  UNKNOWN
Função-objetivo:  5436.0
Best Bound: 716.0000
Nós B&B:  30766111
GAP: 0.8683
CpSolverResponse summary:
status: UNKNOWN
objective: 5436
best_bound: 716
booleans: 190
conflicts: 24369442
branches: 30766111
propagations: 434094206
integer_propagations: 679441273
restarts: 66661
lp_iterations: 0
walltime: 3600.07
usertime: 3600.07
deterministic_time: 6569.51
primal_integral: 55576.6

Lista de Alocação (tarefa, início):


IndexError: ignored

In [ ]:
# *********************** 100 TAREFAS CP-SAT ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("Instancia100.txt")  

print("===== CP-SAT =====")
modelo_Min_TerminoPonderado_CPSAT(N, p, w, d)

===== CP-SAT =====
Status:  UNKNOWN
Função-objetivo:  807748.0
Best Bound: 20868.0000
Nós B&B:  6348809
GAP: 0.9742
CpSolverResponse summary:
status: UNKNOWN
objective: 807748
best_bound: 20868
booleans: 4950
conflicts: 4324978
branches: 6348809
propagations: 700191715
integer_propagations: 634909916
restarts: 23543
lp_iterations: 0
walltime: 3600.05
usertime: 3600.05
deterministic_time: 1150.25
primal_integral: 15615.5

Lista de Alocação (tarefa, início):


IndexError: ignored

In [ ]:
# *********************** 10 TAREFAS CBC ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("instancia10_.txt")

print("===== CBC =====")
modelo_Min_TerminoPonderado_CBC(N, p, w, d)

===== CBC =====
Status:  0
Número de Variáveis:  55
Número de Restrições:  100
Tempo de execução (s): 2.9920
Nós B&B:  219
Função-objetivo:  17481.0
Best Bound: 17481.0000
GAP: 0.0000
Lista de Alocação (tarefa, início):
[(2, 0.0), (7, 79.0), (9, 133.0), (3, 198.0), (0, 253.0), (5, 306.0), (6, 379.0), (4, 507.0), (1, 644.0), (8, 785.0)]



In [ ]:
# *********************** 20 TAREFAS CBC ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("Instancia20.txt")

print("===== CBC =====")
modelo_Min_TerminoPonderado_CBC(N, p, w, d)

===== CBC =====
Status:  6
Número de Variáveis:  210
Número de Restrições:  400
Tempo de execução (s): 3651.1570
Nós B&B:  475883
Função-objetivo:  0.0
Best Bound: 1155.7081
GAP: -1.0000
Lista de Alocação (tarefa, início):
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0)]



In [ ]:
# *********************** 100 TAREFAS CBC ***************************
# Leitura do arquivo com os dados
# N (qtde tarefas), p (proc), w (peso), d (prazo) são as saídas da função leitura
N, p, w, d = leitura("Instancia100.txt")

print("===== CBC =====")
modelo_Min_TerminoPonderado_CBC(N, p, w, d)

===== CBC =====
Status:  6
Número de Variáveis:  5050
Número de Restrições:  10000
Tempo de execução (s): 3781.9360
Nós B&B:  11057
Função-objetivo:  0.0
Best Bound: 23511.5943
GAP: -1.0000
Lista de Alocação (tarefa, início):
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.0), (36, 0.0), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.0), (48, 0.0), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (58, 0.0), (59, 0.0), (60, 0.0), (61, 0.0), (62, 0.0), (63, 0.0), (64, 0.0), (65, 0.0), (66, 0.0), (67, 0.0), (68, 0.0), (69, 0.0), (70, 0.0), (7